# Data understanding
In diesem Notebook werden die für diese Arbeit potentiel wichtigen Informationen in Wikidata explorativ begutachtet.  
Der Code ruf die Daten mittels [Wikidata Query Service](https://query.wikidata.org/) ab.

In diesem Notebooks soll herausgefunden werden, welche Strassen in Basel in Wikidata erfasst sind. Dies hat zwei Ziele:
1. Wie kann der Wikidata Query Service mittels Python angesprochen werden. Dies ist wichtig da später die Strassennamen mit den Einträgen in Wikidata abgeglichen weden sollen.
2. Sind bereits Strassen in Basel in Wikidata erfasst und besitzen die gegebenfalls bereits eine verlinkung zu dem namensgebenden Objekt. Dies könnte interessant sein, da die Informationen für eine Qualitätskontrolle des Abgleichprozesses genutzt werden könnten.

## Strassen mit Wikidata in Basel in OSM
In Wikidata ist es nicht so einfach alle Strassen oder alle Objekte in Basel zu finden, da es keine direkte geografische Abfrage gibt (oder sie ist den Autoren zumindest nicht bekannt). Es kann nur abgefragt werden, welche Strassen ein Statement "liegt in Basel" vergeben wurde, wobei für Basel theoretisch sowohl die Stadt als auch der Kanton verwendet werden kann.  
Deshal wird zuerst überprüft, ob allenfalls in OpenStreetMap bereits Strassen im Kanton Basel mit einem [Wikidata Key](https://wiki.openstreetmap.org/wiki/Key:wikidata) versehen sind.


In [1]:
import requests

#overpass_url = "http://overpass-api.de/api/interpreter"
overpass_url = "https://lz4.overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-2"="CH-BS"];
(
nwr["wikidata"]["highway"](area);
);
out body;
"""

response = requests.get(overpass_url, data = overpass_query)
print(response)

<Response [200]>


In [2]:
data = response.json()
#print(data)

In [3]:
print(type(data))

<class 'dict'>


In [4]:
import pandas as pd
df = pd.json_normalize(data).explode('elements').reset_index(drop=True)
df.head(2)

,version,generator,elements,osm3s.timestamp_osm_base,osm3s.timestamp_areas_base,osm3s.copyright
0,0.6,Overpass API 0.7.58.5 b0c4acbb,"{'type': 'way', 'id': 4303427, 'nodes': [10757...",2022-10-02T18:40:40Z,2022-07-20T10:48:09Z,The data included in this document is from www...
1,0.6,Overpass API 0.7.58.5 b0c4acbb,"{'type': 'way', 'id': 4803840, 'nodes': [30827...",2022-10-02T18:40:40Z,2022-07-20T10:48:09Z,The data included in this document is from www...


endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?nativename ?streetkey WHERE {
				?streetname wdt:P1945 ?streetkey;
				wdt:P131 wd:Q72;
				wdt:P1705 ?nativename.
				SERVICE wikibase:label { bd:serviceParam wikibase:language "de" . } 
				}"""